In [ ]:
#!pip install dgl-cu101
#!pip install dynamicgem
#!pip install keras==2.2.4

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import random
from utils import encode_onehot
from models import RGCN,GCNLSTM,GCN,dgl_GCN,GAT,GraphSage,EGCN,LSTMGCN,RNNGCN,TRNNGCN

import tensorflow
from dynamicgem.embedding.dynAERNN  import DynAERNN

import dgl

import scipy as sp
import scipy.linalg as linalg
import networkx as nx
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans,vq
from scipy import stats  

from sklearn.cluster import SpectralClustering
from sklearn import metrics

from itertools import permutations 

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


# Model

In [ ]:
def one_hot(l,classnum=1): #classnum fix some special case
    one_hot_l=np.zeros((len(l),max(l.max()+1,classnum)))
    for i in range(len(l)):
        one_hot_l[i][l[i]]=1
    return one_hot_l

In [ ]:
def train(epoch, model, optimizer, features, adj, labels, idx_train, idx_val, model_type):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    #print(features.shape)
    output = model(features, adj)
    
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    
    pred_labels=torch.argmax(output,axis=1)
    acc_train = metrics.accuracy_score(pred_labels[idx_train].cpu().detach().numpy(),labels[idx_train].cpu().detach().numpy())
    

    loss_train.backward(retain_graph=True)
    optimizer.step()
    #print(loss_train,acc_train)

    #validation
    model.eval()
    output = model(features, adj)
    
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = metrics.accuracy_score(pred_labels[idx_val].cpu().detach().numpy(),labels[idx_val].cpu().detach().numpy())
    #print(loss_val,acc_val)
    '''
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
    
    a.write('Epoch: {:04d}'.format(epoch+1)+' '+
          'loss_train: {:.4f}'.format(loss_train.item())+' '+
          'acc_train: {:.4f}'.format(acc_train.item())+' '+
          'loss_val: {:.4f}'.format(loss_val.item())+' '+
          'acc_val: {:.4f}'.format(acc_val.item())+' '+
          'time: {:.4f}s'.format(time.time() - t)+'\n')
    a.close()
    '''
    return acc_val

In [ ]:
def test(model, features, adj, labels, idx_test):
    model.eval()
    output = model(features, adj)
    pred_labels=torch.argmax(output,axis=1)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = metrics.accuracy_score(labels[idx_test].cpu().detach().numpy(), pred_labels[idx_test].cpu().detach().numpy())
    f1_test=metrics.f1_score(labels[idx_test].cpu().detach().numpy(), pred_labels[idx_test].cpu().detach().numpy(),average='weighted')
    auc_test=metrics.roc_auc_score(one_hot(labels[idx_test].cpu().detach().numpy()), output[idx_test].cpu().detach().numpy(),multi_class='ovr',average='weighted')
    
    return loss_test.item(), acc_test, f1_test, auc_test

In [ ]:
def getNormLaplacian(W):
	"""input matrix W=(w_ij)
	"compute D=diag(d1,...dn)
	"and L=D-W
	"and Lbar=D^(-1/2)LD^(-1/2)
	"return Lbar
	"""
	d=[np.sum(row) for row in W]
	D=np.diag(d)
	L=D-W
	Dn=np.power(np.linalg.matrix_power(D,-1),0.5)
	Lbar=np.dot(np.dot(Dn,L),Dn)
	return Lbar
 
def getKlargestEigVec(Lbar,k):
	"""input
	"matrix Lbar and k
	"return
	"k largest eigen values and their corresponding eigen vectors
	"""
	eigval,eigvec=linalg.eig(Lbar)
	dim=len(eigval)
 
	#find top k largest eigval
	dictEigval=dict(zip(eigval,range(0,dim)))
	kEig=np.sort(eigval)[::-1][:k]#[0:k]
	ix=[dictEigval[k] for k in kEig]
	return eigval[ix],eigvec[:,ix]
 
def getKlargestSigVec(Lbar,k):
	"""input
	"matrix Lbar and k
	"return
	"k largest singular values and their corresponding eigen vectors
	"""
	lsigvec,sigval,rsigvec=linalg.svd(Lbar)
	dim=len(sigval)
 
	#find top k largest left sigval
	dictSigval=dict(zip(sigval,range(0,dim)))
	kSig=np.sort(sigval)[::-1][:k]#[0:k]
	ix=[dictSigval[k] for k in kSig]
	return sigval[ix],lsigvec[:,ix]

def checkResult(Lbar,eigvec,eigval,k):
	"""
	"input
	"matrix Lbar and k eig values and k eig vectors
	"print norm(Lbar*eigvec[:,i]-lamda[i]*eigvec[:,i])
	"""
	check=[np.dot(Lbar,eigvec[:,i])-eigval[i]*eigvec[:,i] for i in range(0,k)]
	length=[np.linalg.norm(e) for e in check]/np.spacing(1)
	print("Lbar*v-lamda*v are %s*%s" % (length,np.spacing(1)))

In [ ]:
#setting of data generation



def generate_data(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector):
    
    transit_matrix=[]
    for i in range(class_num):
        transit_one=[epsilon_vector[i]]*i+[1-epsilon_vector[i]]+[epsilon_vector[i]]*(class_num-1-i)
        transit_matrix+=[transit_one]
    #print((number_of_nodes*link_inclass_prob*epsilon_vector[0])**0.5)
    
    adj=torch.zeros(number_of_nodes,Time_steps,number_of_nodes) #n*t*n adj matrix

    #assign initial labels
    labels=torch.randint(0,class_num,(number_of_nodes,)) #assign random label with equal probability
    labels=labels.to(dtype=torch.long)
    #label_node, speed up the generation of edges
    label_node_dict=dict()

    for j in range(class_num):
            label_node_dict[j]=[]

    for i in range(len(labels)):
        label_node_dict[int(labels[i])]+=[int(i)]


    #generate graph
    for i in range(int(Time_steps)):
        #change node
        change_nodes=[]
        for j in range(len(labels)):
            if random.random()<epsilon_vector[labels[j]]:
                #less than change probability
                tmp=int(labels[j])
                #print(j)
                while(1): #change label
                    labels[j]=torch.tensor(int(torch.randint(0,class_num,(1,))[0]))
                    if labels[j]!=tmp:
                        change_nodes+=[j]
                        break
                
        label_node_dict=dict()
        for j in range(class_num):
            label_node_dict[j]=[]

        for j in range(len(labels)):
            label_node_dict[int(labels[j])]+=[int(j)]
        #
        #generate symmetrix adj matrix at each time step
        for node_id in range(number_of_nodes):
                j=labels[node_id]
                for l in label_node_dict:
                    if l==j:
                        for z in label_node_dict[l]:  #z>node_id,  symmetrix matrix, no repeat
                            if z>node_id and random.random()<link_inclass_prob:
                                adj[node_id,i,z]= 1
                                adj[z,i,node_id]= 1
                    else:
                        for z in label_node_dict[l]:
                            if z>node_id and random.random()<link_outclass_prob:
                                adj[node_id,i,z]= 1
                                adj[z,i,node_id]= 1
                              


    #generate feature use eye matrix
    features=torch.zeros(number_of_nodes,Time_steps,number_of_nodes)
    for i in range(features.shape[1]):
        features[:,i,:]=torch.eye(features.shape[0],features.shape[2])

    #seprate train,val,test
    idx_train = torch.LongTensor(range(number_of_nodes//5))
    idx_val = torch.LongTensor(range(number_of_nodes//5, number_of_nodes//2))
    idx_test = torch.LongTensor(range(number_of_nodes//2, number_of_nodes))

    #probability matrix at last time_step
    Probability_matrix=torch.zeros(number_of_nodes,number_of_nodes)
    for j in range(number_of_nodes):
        for k in range(number_of_nodes):
          if j==k:
                continue
          elif labels[j]==labels[k]:
            Probability_matrix[j][k]=link_inclass_prob
          else:
            Probability_matrix[j][k]=link_outclass_prob

    return features.float(), adj.float(), labels, idx_train, idx_val, idx_test, Probability_matrix









In [ ]:
def single_train_and_test(lambda_matrix, Probability_matrix, features, adj, labels, idx_train, idx_val, idx_test, model_type,normalize=False):

    if model_type=='SPEC' or model_type=='SPEC_sklearn':
          if type(lambda_matrix)!=type(None):
              decay_adj=torch.zeros(adj.shape[0],adj.shape[2])
              for j in range(adj.shape[0]):
                  for k in range(adj.shape[2]):
                      decay_adj[j][k]=lambda_matrix[labels[j]][labels[k]]
              now_adj=adj[:,0,:].clone()
              for i in range(1,adj.shape[1]):  #time_steps
                          tmp_adj=adj[:,i,:].clone()
                          
                          now_adj=(1-decay_adj)*now_adj+decay_adj*tmp_adj
            
              adj=now_adj
          else:
              now_adj=adj[:,0,:].clone()
              for i in range(1,adj.shape[1]):  #time_steps
                      now_adj+=adj[:,i,:].clone()
              adj=now_adj
          if normalize==True:
              #normalize in both cases
              
              adj+=torch.eye(adj.shape[0],adj.shape[1])
              d=torch.sum(adj,axis=1)
              D_minus_one_over_2=torch.zeros(adj.shape[0],adj.shape[0])
              D_minus_one_over_2[range(len(D_minus_one_over_2)), range(len(D_minus_one_over_2))] = d**(-0.5)
              adj=torch.mm(torch.mm(D_minus_one_over_2,adj),D_minus_one_over_2)

          

        

          Lbar=np.array(adj)  #no normalizaton
          top_k=class_num
          kSigVal,kSigVec=getKlargestSigVec(Lbar,top_k)
          centroid=kmeans(kSigVec.astype(float),class_num)[0] #change kSigvec from complex64 to float
          result=vq(kSigVec.astype(float),centroid)[0]

          
          perm = permutations(range(class_num)) 
          one_hot_result=torch.tensor(one_hot(result,class_num))
          acc_test=0
          f1_test=0
          auc_test=0
          count=0
          for i in perm: 
                count+=1
                one_hot_i=one_hot(np.array(i))
                perm_result=torch.mm(one_hot_result,torch.tensor(one_hot_i))
                pred_labels=torch.argmax(perm_result,axis=1)
                acc_test = max(metrics.accuracy_score(labels,pred_labels),acc_test)
                f1_test=max(metrics.f1_score(labels, pred_labels,average='weighted'),f1_test)
                auc_test=max(metrics.roc_auc_score(one_hot(labels), perm_result,multi_class='ovr',average='weighted'),auc_test)
                if count%10000==0:
                  print(count)
                  print(acc_test,f1_test,auc_test)   
          print(str(acc_test)+'\t'+str(f1_test)+'\t'+str(auc_test))  
          try:
              spec_norm=getKlargestSigVec(adj-Probability_matrix,2)[0]
          except:
              spec_norm=[]
          return 0,acc_test,spec_norm

    elif model_type=="DynAERNN":
        
        length=adj.shape[1]
        lookup=length-2

        dim_emb  = class_num
        if args_cuda:
          tensorflow.device('/gpu:0')
        embedding = DynAERNN(d   = dim_emb,
            beta           = 5,
            n_prev_graphs  = lookup,
            nu1            = 1e-6,
            nu2            = 1e-6,
            n_aeunits      = [50, 30],
            n_lstmunits    = [50,dim_emb],
            rho            = 0.3,
            n_iter         = args_epochs,
            xeta           = 1e-3,
            n_batch        = 10,
            modelfile      = ['./intermediate/enc_model_dynAERNN.json', 
                              './intermediate/dec_model_dynAERNN.json'],
            weightfile     = ['./intermediate/enc_weights_dynAERNN.hdf5', 
                              './intermediate/dec_weights_dynAERNN.hdf5'],
            savefilesuffix = "testing")
        embs = []
        
        graphs     = [nx.Graph(adj[:,l,:].numpy()) for l in range(length)]
        for temp_var in range(lookup, length):
                        emb, _ = embedding.learn_embeddings(graphs[:temp_var])
                        embs.append(emb)
        centroid=kmeans(embs[-1],class_num)[0] #change kSigvec from complex64 to float
        result=vq(embs[-1],centroid)[0]

        

        perm = permutations(range(class_num)) 
        one_hot_result=torch.tensor(one_hot(result,class_num))
        acc_test=0
        f1_test=0
        auc_test=0
        count=0
        for i in perm: 
              count+=1
              one_hot_i=one_hot(np.array(i))
              perm_result=torch.mm(one_hot_result,torch.tensor(one_hot_i))
              pred_labels=torch.argmax(perm_result,axis=1)
              acc_test = max(metrics.accuracy_score(labels,pred_labels),acc_test)
              f1_test=max(metrics.f1_score(labels, pred_labels,average='weighted'),f1_test)
              auc_test=max(metrics.roc_auc_score(one_hot(labels), perm_result,multi_class='ovr',average='weighted'),auc_test)
              if count%10000==0:
                print(count)
                print(acc_test,f1_test,auc_test)   
        print(str(acc_test)+'\t'+str(f1_test)+'\t'+str(auc_test))  
        try:
              spec_norm=getKlargestSigVec(adj-Probability_matrix,2)[0]
        except:
              spec_norm=[]
        return 0,acc_test,spec_norm
        


    #choose adj matrix
    #GCN:n*n, Others: n*t*n
    if model_type=='GCN':  
          if type(lambda_matrix)!=type(None):
            decay_adj=torch.zeros(adj.shape[0],adj.shape[0])
            for j in range(adj.shape[0]):
                for k in range(adj.shape[2]):
                    decay_adj[j][k]=lambda_matrix[labels[j]][labels[k]]
            now_adj=adj[:,0,:].clone()
            
            for i in range(1,adj.shape[1]):  #time_steps
                    tmp_adj=adj[:,i,:].clone()
                    
                    now_adj=(1-decay_adj)*now_adj+decay_adj*tmp_adj
            adj=now_adj
          else:
              now_adj=adj[:,0,:].clone()
              for i in range(1,adj.shape[1]):  #time_steps
                      now_adj+=adj[:,i,:].clone()
              adj=now_adj
              
          #normalize in both cases
          if normalize==True:
              adj+=torch.eye(adj.shape[0],adj.shape[1])
              d=torch.sum(adj,axis=1)
              D_minus_one_over_2=torch.zeros(adj.shape[0],adj.shape[0])
              D_minus_one_over_2[range(len(D_minus_one_over_2)), range(len(D_minus_one_over_2))] = d**(-0.5)
              adj=torch.mm(torch.mm(D_minus_one_over_2,adj),D_minus_one_over_2)
              
          
          features=features[:,-1,:]
          

    elif model_type=='GAT' or model_type=='GraphSage':
          now_adj=adj[:,0,:].clone()
          for i in range(1,adj.shape[1]):  #time_steps
                  now_adj+=adj[:,i,:].clone()
          adj=now_adj
          
          #normalize in both cases
          if normalize==True:
              adj+=torch.eye(adj.shape[0],adj.shape[1])
              d=torch.sum(adj,axis=1)
              D_minus_one_over_2=torch.zeros(adj.shape[0],adj.shape[0])
              D_minus_one_over_2[range(len(D_minus_one_over_2)), range(len(D_minus_one_over_2))] = d**(-0.5)
              adj=torch.mm(torch.mm(D_minus_one_over_2,adj),D_minus_one_over_2)
              
          features=features[:,-1,:]
    elif model_type=='EGCN':
        adj=torch.transpose(adj,0,1)
        features=torch.transpose(features,0,1)
        

    #define model
    if model_type=='GCN':
        model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=='RNNGCN':
        model = RNNGCN(nfeat=features.shape[2],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=='TRNNGCN':
        model = TRNNGCN(nfeat=features.shape[2],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout,
                nnode=features.shape[0],
                use_cuda=args_cuda)
    elif model_type=='GCNLSTM':
        model = GCNLSTM(nfeat=features.shape[2],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=='RGCN':
        model = RGCN(nfeat=features.shape[2],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=="GAT":
        adj=dgl.from_networkx(nx.Graph(adj.numpy())) #fit in dgl
        model = GAT(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=="GraphSage":
        adj=dgl.from_networkx(nx.Graph(adj.numpy())) #fit in dgl
        model = GraphSage(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=class_num,
                dropout=args_dropout)
    elif model_type=="EGCN":
        model = EGCN(nfeat=features.shape[2],
                    nhid=args_hidden,
                    nclass=class_num,
                    device=torch.device('cpu'))

    
        
    if model_type!="SPEC" and model_type!="SPEC_sklearn" and model_type!="DynAERNN":
        if args_cuda:
            if model_type!='EGCN':
                model=model.to(torch.device('cuda:0'))#.cuda()
                features = features.cuda()
                adj = adj.to(torch.device('cuda:0'))
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizer = optim.Adam(model.parameters(),
                              lr=args_lr, weight_decay=args_weight_decay)
        # Train model
        t_total = time.time()
        best_val=0
        for epoch in range(args_epochs):
            acc_val=train(epoch, model, optimizer, features, adj, labels, idx_train, idx_val, model_type)
            #print(model.Lambda)
            if acc_val>best_val:
              best_val=acc_val
              loss, acc, auc, f1 = test(model, features, adj, labels, idx_test)
              test_best_val=[loss,acc,auc,f1]
            
        # Testing
        loss, acc, auc, f1 = test(model, features, adj, labels, idx_test)
        if model_type=='RNNGCN' or model_type=='TRNNGCN':
          print(model.Lambda,end='\t')
        #print(loss,acc)
        print(str(test_best_val[1])+'\t'+str(test_best_val[2])+'\t'+str(test_best_val[3]))#,end='\t')
        try:
            spec_norm=getKlargestSigVec(now_adj-Probability_matrix,2)[0]
        except:
            spec_norm=0 #temperal adj
        return loss, acc, spec_norm

# Run Exp for Spectral Clustering and GCN with Decay Rates

In [ ]:
def test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time):  
     for times in range(sample_time):     
          try:
              features, adj, labels, idx_train, idx_val, idx_test, Probability_matrix=generate_data(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector)               
              for i in np.arange(0.0, 1.01, 0.01):
                        file_name='uncombined'+'_'+model_type+"_" +"number_of_nodes_"+str(number_of_nodes)+'_' +"Time_steps_"+str(Time_steps)+'_'\
                                      +"class_num_"+str(class_num)+'_' +"link_inclass_prob_"+str(link_inclass_prob)+'_'\
                                      +"link_outclass_prob_"+str(link_outclass_prob)+'_'+"epsilon_vector_"+str(epsilon_vector)+'_'\
                                      +"sample_time_"+str(sample_time)+".txt"
                        if IN_COLAB==True:
                            summary_file = open("/content/drive/My Drive/"+file_name,"a+")
                        else:
                            summary_file = open(file_name,"a+")
                        t=time.time()
                        lambda_matrix=np.full((class_num,class_num),i)
                        
                        total_loss=0
                        total_acc=0
                        total_norm=[]
                        loss, acc, specnorm = single_train_and_test(lambda_matrix,Probability_matrix,features, adj, labels, idx_train, idx_val, idx_test, model_type)

                        summary_file.write("Weight decay: {}".format(lambda_matrix.flatten()) +
                                "\tTest set results:" +
                                "\tloss= {:.6f}".format(loss) + 
                                "\taccuracy= {:.6f}".format(acc)+
                                "\tspecnorm= {}\n".format(specnorm))
                        summary_file.close()
          except:
            error=1
                
def test_epsilon_vector_kxklambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time):  
     for times in range(sample_time):     
          try:
              features, adj, labels, idx_train, idx_val, idx_test, Probability_matrix=generate_data(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector)               
              for i in np.arange(0.0, 1.01, 0.1):
                for j in np.arange(0, 1.01, 0.1):
                  for k in np.arange(0, 1.01, 0.1):
                    for l in np.arange(0, 1.01, 0.1):
                        file_name='uncombined'+'_'+'kxklambda'+'_'+model_type+"_" +"number_of_nodes_"+str(number_of_nodes)+'_' +"Time_steps_"+str(Time_steps)+'_'\
                                      +"class_num_"+str(class_num)+'_' +"link_inclass_prob_"+str(link_inclass_prob)+'_'\
                                      +"link_outclass_prob_"+str(link_outclass_prob)+'_'+"epsilon_vector_"+str(epsilon_vector)+'_'\
                                      +"sample_time_"+str(sample_time)+".txt"
                        if IN_COLAB==True:
                            summary_file = open("/content/drive/My Drive/"+file_name,"a+")
                        else:
                            summary_file = open(file_name,"a+")
                        t=time.time()
                        lambda_matrix=np.array([[i,j],[k,l]])
                        total_loss=0
                        total_acc=0
                        total_norm=[]
                        loss, acc, specnorm = single_train_and_test(lambda_matrix,Probability_matrix,features, adj, labels, idx_train, idx_val, idx_test, model_type)

                        summary_file.write("Weight decay: {}".format(lambda_matrix.flatten()) +
                                "\tTest set results:" +
                                "\tloss= {:.6f}".format(loss) + 
                                "\taccuracy= {:.6f}".format(acc)+
                                "\tspecnorm= {}\n".format(specnorm))
                        
                        summary_file.close()
          except:
            error=1
            
def test_kxk_neural_network(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time):  
     for times in range(sample_time):     
              features, adj, labels, idx_train, idx_val, idx_test, Probability_matrix=generate_data(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector)               
              for i in range(1):
                        file_name='uncombined'+'_'+'kxklambda'+'_'+model_type+"_" +"number_of_nodes_"+str(number_of_nodes)+'_' +"Time_steps_"+str(Time_steps)+'_'\
                                      +"class_num_"+str(class_num)+'_' +"link_inclass_prob_"+str(link_inclass_prob)+'_'\
                                      +"link_outclass_prob_"+str(link_outclass_prob)+'_'+"epsilon_vector_"+str(epsilon_vector)+'_'\
                                      +"sample_time_"+str(sample_time)+".txt"
                        if IN_COLAB==True:
                            summary_file = open("/content/drive/My Drive/"+file_name,"a+")
                        else:
                            summary_file = open(file_name,"a+")
                        t=time.time()
                        lambda_matrix=np.full((class_num,class_num),0.2)
                        #print("current matrix: {}".format(lambda_matrix))
                        total_loss=0
                        total_acc=0
                        total_norm=[]
                        loss, acc, specnorm = single_train_and_test(lambda_matrix,Probability_matrix,features, adj, labels, idx_train, idx_val, idx_test, model_type)
                        
                        summary_file.write("Weight decay: {}".format(lambda_matrix.flatten()) +
                                "\tTest set results:" +
                                "\tloss= {:.6f}".format(loss) + 
                                "\taccuracy= {:.6f}".format(acc)+
                                "\tspecnorm= {}\n".format(specnorm))
                        print(i,loss,acc,specnorm)
                        #print(time.time()-t)
                        summary_file.close()


                        
#For simulated graphs

sample_time=100
number_of_nodes=200
Time_steps=500
class_num=2
link_inclass_prob=20/number_of_nodes/5  #when calculation , remove the link in itself

link_outclass_prob=link_inclass_prob/20
epsilon_vector=[10/number_of_nodes,20/number_of_nodes]



model_type='SPEC'    #GCN, GAT, GraphSage #SPEC(DynSPEC), DynAERNN #GCNLSTM, EGCN, RNNGCN, TRNNGCN
args_hidden = class_num
args_dropout = 0.5
args_lr = 0.01
args_weight_decay = 5e-4
args_epochs = 250
args_no_cuda=False
args_cuda = not args_no_cuda and torch.cuda.is_available()







###Different setting on simulated graphs

#test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)



#for number_of_nodes in [100,250,500]:
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for link_inclass_prob in [10/number_of_nodes/5,20/number_of_nodes/5,30/number_of_nodes/5]:
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for epsilon_vector in [[10/number_of_nodes,10/number_of_nodes],[20/number_of_nodes,20/number_of_nodes],[30/number_of_nodes,30/number_of_nodes],[40/number_of_nodes,40/number_of_nodes],[50/number_of_nodes,50/number_of_nodes],[60/number_of_nodes,60/number_of_nodes]]:
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for Time_steps in [1000,2000,5000,10000]: #already have 500
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for sample_time in [1,10,1000]: #already have 100
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for epsilon_vector in [[10/number_of_nodes,20/number_of_nodes],[10/number_of_nodes,30/number_of_nodes],[20/number_of_nodes,30/number_of_nodes]]:
#    test_epsilon_vector_onelambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)

#for epsilon_vector in [[10/number_of_nodes,20/number_of_nodes],[10/number_of_nodes,30/number_of_nodes],[20/number_of_nodes,30/number_of_nodes]]:
#for epsilon_vector in [[10/number_of_nodes,40/number_of_nodes],[20/number_of_nodes,40/number_of_nodes],[30/number_of_nodes,40/number_of_nodes]]:
#    test_epsilon_vector_kxklambda(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)


#test_kxk_neural_network(model_type,number_of_nodes,Time_steps,class_num,link_inclass_prob,link_outclass_prob, epsilon_vector,sample_time)


# Run Exp on Simulated and Real Datasets

In [ ]:
def load_real_data(dataset_name):
    dataset_dict=dict()
    dataset_dict["DBLP3"]="DBLP3.npz"
    dataset_dict["DBLP5"]="DBLP5.npz"
    dataset_dict["Brain"]="Brain.npz"
    dataset_dict["Reddit"]="reddit.npz"
    dataset_dict["DBLPE"]="DBLPE.npz"
    
    dataset      = np.load(dataset_dict[dataset_name])
    
    
    Graphs    = torch.LongTensor(dataset['adjs'])    #(n_time, n_node, n_node)
    Graphs=torch.transpose(Graphs,0,1) #(n_node, n_time, n_node)

    now_adj=Graphs[:,0,:].clone()
    for i in range(1,Graphs.shape[1]):  #time_steps
                  now_adj+=Graphs[:,i,:].clone()
    d=torch.sum(now_adj,axis=1)
    non_zero_index=torch.nonzero(d,as_tuple=True)[0]
    Graphs=Graphs[non_zero_index,:,:]
    Graphs=Graphs[:,:,non_zero_index]
    

    if dataset_name=="DBLPE":
      Labels = torch.LongTensor(np.argmax(dataset['labels'],axis=2))  #(n_node, n_time, num_classes) argmax
      Features=torch.zeros(Graphs.shape)
      for i in range(Features.shape[1]):
          Features[:,i,:]=torch.eye(Features.shape[0],Features.shape[2])
      Labels=Labels[non_zero_index]
      
    else:
      Labels    = torch.LongTensor(np.argmax(dataset['labels'],axis=1))  #(n_node, num_classes) argmax
      Features  = torch.LongTensor(dataset['attmats']) #(n_node, n_time, att_dim)
  
      Features=Features[non_zero_index]
      Labels=Labels[non_zero_index]
    

    
    #shuffle datasets
    number_of_nodes=Graphs.shape[0]
    nodes_id=list(range(number_of_nodes))
    random.shuffle(nodes_id)
    idx_train = torch.LongTensor(nodes_id[:(7*number_of_nodes)//10])
    idx_val = torch.LongTensor(nodes_id[(7*number_of_nodes)//10: (9*number_of_nodes)//10])
    idx_test = torch.LongTensor(nodes_id[(9*number_of_nodes)//10: number_of_nodes])
    
    return Features.float(), Graphs.float(), Labels.long(), idx_train, idx_val, idx_test, []

In [ ]:
def test_real_dataset():
                  
    file_name=dataset_name+'_'+model_type+".txt"
    if IN_COLAB==True:
        summary_file = open("/content/drive/My Drive/"+file_name,"a+")
    else:
        summary_file = open(file_name,"a+")
    t=time.time()
    lambda_matrix=None 
    total_loss=0
    total_acc=0
    total_norm=[]
    loss, acc, specnorm = single_train_and_test(lambda_matrix,Probability_matrix,features, adj, labels, idx_train, idx_val, idx_test, model_type,normalize=args_normalize)
    if type(lambda_matrix)!=type(None):
        summary_file.write("Weight decay: {}".format(lambda_matrix.flatten()) +
                                    "\tTest set results:" +
                                    "\tloss= {:.6f}".format(loss) + 
                                    "\taccuracy= {:.6f}".format(acc)+
                                    "\tspecnorm= {}\n".format(specnorm))
    else:
        summary_file.write("Weight decay: {}".format(0) +
                                    "\tTest set results:" +
                                    "\tloss= {:.6f}".format(loss) + 
                                    "\taccuracy= {:.6f}".format(acc)+
                                    "\tspecnorm= {}\n".format(specnorm))
    
    summary_file.close()

In [ ]:
#simulated data: setting of data generation

def generate_data_totallabel(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector):
    
    transit_matrix=[]
    for i in range(class_num):
        transit_one=[epsilon_vector[i]]*i+[1-epsilon_vector[i]]+[epsilon_vector[i]]*(class_num-1-i)
        transit_matrix+=[transit_one]
    #print((number_of_nodes*link_inclass_prob*epsilon_vector[0])**0.5)
    
    adj=torch.zeros(number_of_nodes,Time_steps,number_of_nodes) #n*t*n adj matrix

    #assign initial labels
    labels=torch.randint(0,class_num,(number_of_nodes,)) #assign random label with equal probability
    labels=labels.to(dtype=torch.long)
    #label_node, speed up the generation of edges
    label_node_dict=dict()

    for j in range(class_num):
            label_node_dict[j]=[]

    for i in range(len(labels)):
        label_node_dict[int(labels[i])]+=[int(i)]

    total_labels=torch.zeros(number_of_nodes,Time_steps)
    #generate graph
    for i in range(int(Time_steps)):
        #change node
        change_nodes=[]
        for j in range(len(labels)):
            if random.random()<epsilon_vector[labels[j]]:
                #less than change probability
                tmp=int(labels[j])
                #print(j)
                while(1): #change label
                    labels[j]=torch.tensor(int(torch.randint(0,class_num,(1,))[0]))
                    if labels[j]!=tmp:
                        change_nodes+=[j]
                        break
                #labels[j]=torch.tensor(not tmp)
        total_labels[:,i]=labels.clone()
        label_node_dict=dict()
        for j in range(class_num):
            label_node_dict[j]=[]

        for j in range(len(labels)):
            label_node_dict[int(labels[j])]+=[int(j)]
        #
        #generate symmetrix adj matrix at each time step
        for node_id in range(number_of_nodes):
                j=labels[node_id]
                for l in label_node_dict:
                    if l==j:
                        for z in label_node_dict[l]:  #z>node_id,  symmetrix matrix, no repeat
                            if z>node_id and random.random()<link_inclass_prob:
                                adj[node_id,i,z]= 1
                                adj[z,i,node_id]= 1
                    else:
                        for z in label_node_dict[l]:
                            if z>node_id and random.random()<link_outclass_prob:
                                adj[node_id,i,z]= 1
                                adj[z,i,node_id]= 1
                              


    #generate feature use eye matrix
    features=torch.zeros(number_of_nodes,Time_steps,number_of_nodes)
    for i in range(features.shape[1]):
        features[:,i,:]=torch.eye(features.shape[0],features.shape[2])

    #seprate train,val,test
    idx_train = torch.LongTensor(range(number_of_nodes//5))
    idx_val = torch.LongTensor(range(number_of_nodes//5, number_of_nodes//2))
    idx_test = torch.LongTensor(range(number_of_nodes//2, number_of_nodes))

    #probability matrix at last time_step
    Probability_matrix=torch.zeros(number_of_nodes,number_of_nodes)
    for j in range(number_of_nodes):
        for k in range(number_of_nodes):
          if j==k:
                continue
          elif labels[j]==labels[k]:
            Probability_matrix[j][k]=link_inclass_prob
          else:
            Probability_matrix[j][k]=link_outclass_prob

    return features.float(), adj.float(), total_labels.long(), idx_train, idx_val, idx_test, Probability_matrix


In [ ]:
mode="real"

if mode=='real':
    dataset_name="DBLPE"
    features, adj, labels, idx_train, idx_val, idx_test, Probability_matrix=load_real_data(dataset_name) 

    class_num=int(labels.max())+1
    print(class_num)
    total_adj=adj
    total_labels=labels
elif mode=='simulate':
    dataset_name=''
    number_of_nodes=200
    Time_steps=50
    class_num=2
    link_inclass_prob=20/number_of_nodes/5  #when calculation , remove the link in itself
    #EGCN good when network is dense 20/number_of_nodes  #fails when network is sparse. 20/number_of_nodes/5

    link_outclass_prob=link_inclass_prob/20
    epsilon_vector=[10/number_of_nodes,20/number_of_nodes]


    features, adj, labels, idx_train, idx_val, idx_test, Probability_matrix=generate_data_totallabel(number_of_nodes, Time_steps, class_num, link_inclass_prob, link_outclass_prob, epsilon_vector)               
    total_adj=adj
    total_labels=labels

In [ ]:
model_type='TRNNGCN'    #GCN, GAT, GraphSage #dynamic_spec, DynAERNN #GCNLSTM, EGCN, RNNGCN, TRNNGCN
args_hidden = class_num
args_dropout = 0.5
args_lr = 0.0025
args_weight_decay = 5e-4
args_epochs = 500 
args_no_cuda=True
args_cuda = not args_no_cuda and torch.cuda.is_available()
args_normalize=True

if mode=='real':
    if dataset_name=="DBLPE":
      #target_time=13 #0-13
      for target_time in range(0,14):
          print(target_time,end='\t')
          adj = total_adj[:,:target_time+1,:]
          labels = total_labels[:,target_time]
          test_real_dataset()
          print(' ',end='\n')
    else:
        test_real_dataset()
elif mode=='simulate':
    for target_time in range(0,total_labels.shape[1]):
          print(target_time,end='\t')
          adj = total_adj[:,:target_time+1,:]
          labels = total_labels[:,target_time]
          test_real_dataset()
          print(' ',end='\n')